# User Data ETL
## Extract, clean and load User data to SQL Sales database

In [1]:
import sys
sys.path.append('/Users/ronakasher/Deepa/multinational_retail_data_centralisation')

In [2]:
from source_code.database_utils import DatabaseConnector
from source_code.data_extraction import DataExtractor
from source_code.data_cleaning import DataCleaning
import pandas as pd

## Extract data from SQLAlchemy DB using yaml credentials file

In [3]:
# Read credentials from yaml file and return dictionary of credentials
connector = DatabaseConnector('../db_creds.yaml')
db_creds = connector.read_db_creds()

In [4]:
#Initialise and return sqlalchemy db engine
db_engine = connector.init_db_engine()

In [5]:
#List all tables in DB
table_list = connector.list_db_tables()
print(table_list)

['legacy_store_details', 'legacy_users', 'orders_table']


In [6]:
#Extract user database table to pandas DataFrame
extractor = DataExtractor()
user_data = extractor.read_rds_table('legacy_users')
user_data.sample(100)

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
568,Roy,Thomas,1988-05-20,Allen and Sons,katherine50@cook.com,Flat 95\nChamberlain valley\nNew Oliverside\nO...,United Kingdom,GB,+44(0)191 4960408,2013-01-11,792f3252-f82a-4cb9-b350-6a2e5170cef0
6619,Katy,Röhrdanz,1966-08-07,Wernecke,gkuehnert@kreusel.net,Schwitalallee 77\n97251 Hettstedt,Germany,DE,00462741847,1997-04-30,78b4eef9-a21b-4b30-a85a-a575a8edc93d
7675,Katie,Scott,1991-02-01,"Davison, Green and Palmer",dorothyjackson@walker.org,Flat 64z\nO'Connor pass\nMandybury\nM3B 6DJ,United Kingdom,GB,+44(0)808 157 0087,1997-07-05,d3d6e4ee-bfde-4b61-b1b0-02b352b55312
6060,Elliott,Kirk,1950-01-21,Dickinson Group,sjohnston@williams.com,07 Norton village\nO'Connorfort\nN9 7AW,United Kingdom,GB,01134960911,2007-08-30,b18a1c47-8fae-41ef-a429-ce3c5f19c51f
11864,Nelly,Grein Groth,1981-02-22,Kensy,hans-martin63@butte.com,Anatolij-Stroh-Platz 86\n39137 Dinkelsbühl,Germany,DE,+49 (0) 4450 526080,1997-02-17,6c0d08c9-f753-4ba9-b90f-e515301c8903
...,...,...,...,...,...,...,...,...,...,...,...
12184,Charlotte,Brown,2002-09-01,Thompson-Myers,ellie80@davies.biz,Studio 8\nJemma dale\nWest Katherineview\nW16 9TJ,United Kingdom,GB,+44116 4960558,1996-06-09,1476d07c-25b4-41e4-b866-09d02cd5f3fc
951,Jayne,Vincent,1952-04-24,"Wilson, Doyle and Foster",kimberley11@king-gibbs.com,91 Singh trail\nSchofieldport\nB24 7SR,United Kingdom,GB,03069990070,2007-12-15,52a82d6a-2750-4c03-8100-89f4982f584d
3437,Kayleigh,Russell,1969-09-30,Wilson Group,taylorrussell@connolly-matthews.net,Flat 6\nHawkins ports\nMyersberg\nN35 1XA,United Kingdom,GB,+44117 4960572,2002-07-05,2774c1fa-91d6-4c34-ab0d-148d188e8027


In [7]:
user_data[user_data.duplicated(subset='user_uuid',keep=False)]

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
867,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1023,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1807,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2103,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2439,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
6526,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2764,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4987,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
5310,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


## Clean User data using DataCleaning class from source code

In [8]:
#Clean user data
clean_user_data = DataCleaning().clean_user_data(user_data)

In [9]:
#Remove Null rows in cleaned data
clean_user_data.dropna(how='all')
clean_user_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15300 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15284 non-null  object
 1   last_name      15285 non-null  object
 2   date_of_birth  15284 non-null  object
 3   company        15284 non-null  object
 4   email_address  15241 non-null  string
 5   address        15300 non-null  object
 6   country        15284 non-null  object
 7   country_code   15278 non-null  string
 8   phone_number   15300 non-null  object
 9   join_date      15284 non-null  object
 10  user_uuid      15284 non-null  object
dtypes: object(9), string(2)
memory usage: 1.4+ MB


In [10]:
#Upload to sales_data DB (SQL)
upload = DatabaseConnector('../db_creds.yaml')
upload.upload_to_db(clean_user_data,'dim_users')


300